# Application
This is about exploratory data analysis of product_a.csv. This is the only dataset this customer has agreed to expose. The customer would not provide more information until we produce some insights.
Add headers or comments with the complete questions asked in the code or jupyter notebook so that we can locate your approach efficiently.dex.

In [ ]:
# Installing 
!pip install ydata-profiling  # Installing ydata profiling as pandas profiling is deprecated

In [ ]:
# Import libraries
import pandas as pd
import plotly.express as px
import kaleido
import plotly.graph_objs as go
from ydata_profiling import ProfileReport

## 1. Question 1
Import product_a.csv dataset into python pandas data frame df_product_a. The first column is the index.

In [ ]:
# Import the csv file taking the first column as index
#data = pd.read_csv('C:/Iresha_Code/module2/product_a.csv',index_col=0)
data = pd.read_csv('C:/Iresha_Code/module2/product_a.csv')

# Create a pandas dataframe
df_original = pd.DataFrame(data)
df_original

## Question 2
Convert date_w field to a suitable datetime data type.

In [ ]:
# To check the existing data type
df_original['date_w'].dtype

In [ ]:
# To convert to a datetime data type
df_original['date_w']=pd.to_datetime(df_original['date_w'])


# Rename the df, after data type correction
df_c1=df_original
df_c1.head()

In [ ]:
# Verify data type change
df_c1['date_w'].dtype

## Additional - data check 1
To get a better understanding of the dataset, I filtered for one location and sorted by the date_w column. The results indicate
- date_w values are 7 days apart
- Each record gives unit price, total_vol, plu volumes, bag sizes and type for each location, for each day/week.

In [ ]:
# Filter rows for Albany location
df_Alb=df_c1[df_c1['location']== 'Albany']

# Sort by date_w
df_Alb_sort=df_Alb.sort_values('date_w')
df_Alb_sort

## Question 3
Values of the year column do not match with the values of the date_w column. Correct the values of the year column.

In [ ]:
# Extract year from the date_w column and use that to replace values in the year column
df_c1['year']=df_c1['date_w'].dt.year

# Rename df after year column correction
df_c2=df_c1
df_c2.head()

## Additional - data check 2
Apart from the changes to date_w and year columns, I did some further data chekcking/cleaning before creating statistics.  
### Check for any missing values
- After running dropna, the number of rows has not changed, still 18,249. So there are no missing values.

In [ ]:
# Checking for columns with missing values
df_c2.dropna()

#### Checking data types

In [ ]:
# Check ing data types of all fields
df_c2.dtypes

In [ ]:
# Changing data types of volume and bag size fiels to integer 
df_c2[['total_vol','plu1','plu2','plu3','bags_t','bags_s','bags_l','bags_lx']]=df_c2[['total_vol','plu1','plu2','plu3','bags_t','bags_s','bags_l','bags_lx']].astype(int)



In [ ]:
# Verify change of data types
df_c2.dtypes

#### Checking whether totals match

In [ ]:
# Checking whether total bags(bags_t) is the sum of small, large and lx bags. If false all rows match. 

not(df_c2['bags_t'] == df_c2['bags_s']+ df_c2['bags_l']+ df_c2['bags_lx']).all()


In [ ]:
# Correcting bags_t

df_c2['bags_t']= df_c2['bags_s']+ df_c2['bags_l']+ df_c2['bags_lx']
df_c2

#### Checking unique values of columns

In [ ]:
# Check type
unique_type=df_c2['type'].unique()
unique_type

In [ ]:
# Check location
unique_location=df_c2['location'].unique()
unique_location

#### Removing some rows and columns before calculating statistics

In [ ]:
# Removing rows where location is TotalUS
df_c3=df_c2[df_c2['location']!='TotalUS']
df_c3

In [ ]:
# Removing 2nd column
df_c4=df_c3.drop(columns=['Unnamed: 0'])
df_c4

## Question 4
For the numeric columns, create df_stats with the following details from df_product_a.
Columns of df_stats:
 field_name, minimum, maximum, mean, standard deviation, variance, mode, median, 0th, 10th, 20th ... 90th, 100th percentiles, 1st, 2nd and 3rd quartiles, interquartile distance, skewness and kurtosis.

Theory: Discuss the relationships between the fields of df_stats. For example, the 2nd quartile and the median are the same.
Discuss how the columns of df_stats are useful in data analysis.
Analyse the data based on your discussion and explain the results. What are the notable features of the dataset?

### Calculating statistics

In [ ]:
# Create a new dataframe to include only number fields
df_numeric=df_c4.select_dtypes(include=['number'])
df_numeric.head()

In [ ]:
# Calculate statistics
min=df_numeric.min()
max=df_numeric.max()
mean=df_numeric.mean()
std=df_numeric.std()
var=df_numeric.var()
median=df_numeric.median()
q1=df_numeric.quantile(0.25)
q2=df_numeric.quantile(0.5)
q3=df_numeric.quantile(0.75)
iqr=q3-q1
skew=df_numeric.skew()
kurt=df_numeric.kurt()

# Create a new dataframe for statistics
df_stats= pd.DataFrame({
    'min':min,
    'max':max,
    'mean':mean,
    'median':median,
    'std':std,
    'var':var,
    'q1':q1,
    'q2':q2,
    'q3':q3,
    'iqr':iqr,
    'skew':skew,
    'kurt':kurt
    
})

df_stats


In [ ]:
mod=df_numeric.mode()  # The output suggests that all fields are uni-modal expect for total_vol which is tri-modal
mod  

### Analysis:
#### Relationship between fields of statistics
- Median is the same as the 2nd quartile (q2).
- Both var and std measure spread from the mean.
- Std is the square root of var.
- Interquartile range, iqr = q3-q1
- A positive skew means that the mean>median>mode.

#### How the columns are useful for data analysis
- The difference between max and min give the range.
- Mean is the average.
- When the values and ordered ascending, median is the central value.
- q1 indicates 25%, q2 50% and q3 75%. 
- A positive skew means that the mean>median>mode. 
- A negative skew means mean<median<mode.
- If kurt>3, this means the distribution has a heavy tail and a sharp peak, compared to a normal distribution.
- If kurt<3, the distribution has light tails and a flat peak, compared to a normal distribution.

#### Noteable features of the dataset
- The unit price is mostly concentrated around the mean with few outliers (as kurt is closer to zero, light tail).
- However, all the other variables plu1, plu2, plu3, bags_t, bags_s, bags_l, bags_lx indicate having extreme values (as kurt >> 3). 
- In terms of the sizes of bags, small bags are used more, with an average of 116k (per week)  compared to 34k for large bags and 2k for extra large bags. (comparing the mean values)
- Further, extra large bag sizes were not used in a majority of weeks (as q2 value is zero).
- The variables plu1, plu2, plu3, bags_t, bags_s, bags_l, bags_lx all have zero mode, which could be the reason for all those variables having a positve skew. This needs to be further investigated.

## Question 5
### Pearson correlation matrix
Calculate the Pearson correlation matrix (it is a square matrix) between all the possible fields. What are the conclusions yo make??

In [ ]:
# Select fields
col_p=['price', 'total_vol', 'plu1', 'plu2', 'plu3', 'bags_t', 'bags_s', 'bags_l', 'bags_lx']

# Calculate pearson correlation matrix
corr_matrix= df_numeric[col_p].corr(method='pearson')
corr_matrix

Conclusions:
- If the unit price goes up, the other fields of total volume, the volume of each plu, total bag size and each individual bag size per transaction could slightly go down. If the unit price goes down, the same fields could slights go up. However, this increase or decrease is not strong or consistent.  
- There is a extremely strong relationship between the small bags and the total bags. When small bags increase there is a very high change that total bags also increase, and vice versa.
- Also, when there is another strong relationship betweem small bags and total volume. When small bags increase there is a very high change that total volume also increase, and vice versa.
- This suggests that when there is an increase or decrease in total colume and the total bags, small bags would also increase or decrease.

### Spearman correlation matrix
Calculate Spearman’s Rank correlation matrix (it is a square matrix) between all the possible fields. What are the conclusions you make?

In [ ]:
# Select fields
col_p=['price', 'total_vol', 'plu1', 'plu2', 'plu3', 'bags_t', 'bags_s', 'bags_l', 'bags_lx']
corr_r_matrix= df_numeric[col_p].corr(method='spearman')
corr_r_matrix

Conclusions
- Price has a linear inverse relationship with other variables, that is when price goes up, the other volume and bag size related variables go down and vice versa. The strength of thsse relationships are moderate.
- There is a extremely strong relationship between the small bags and the total bags. When small bags increase there is a very high change that total bags also increase, and vice versa.
- Also, when there is another strong relationship betweem small bags and total volume. When small bags increase there is a very high change that total volume also increase, and vice versa.
- This suggests that when there is an increase or decrease in total colume and the total bags, small bags would also increase or decrease.

## Question 6
Draw a Plotly scatter matrix plot for df_product_a. What are the conclusions you can make using the analysis so far?

In [ ]:
# Data
df_c4.head()

In [ ]:
# Create a scatter matrix for price and volume related data
fig1=px.scatter_matrix(df_c4, dimensions=['price', 'total_vol', 'plu1', 'plu2', 'plu3'], title='Scatter Matrix for price and volumes')

# Create a scatter matrix for price and bag size related data
fig2=px.scatter_matrix(df_c4, dimensions=['price', 'bags_s', 'bags_l', 'bags_lx', 'bags_t'], title='Scatter Matrix for price and bag sizes')

In [ ]:
fig1.write_image('ScatterMatrix1_ProductA.png')
fig2.write_image('ScatterMatrix2_ProductA.png')

Conclusions:
- When considering volumes,
    - The variation of total and individual plu volumes against unit price is similar. For lower prices, there are higher volumes, but the volumes go down when the unit price goes up.
    - When total volume goes up, plu 1 and plu2 show a generally increasing trend, but the relationship between total volume and plu3 is not that clear. This suggests that plu1 and plu2 are stronger predictors of total volume than plu 3.
  
- When considering bag sizes,
    - The variation of the 3 bag sizes and the total bag size against unit price is similar.  For lower prices, there are higher number of bag sizes used, but the volumes go down when the unit prices go up.
    - This relationship between bag sizes and price is similar to the one with volumes and price. Both are inverse relationships. 
    - When total bags increase, small bags and large bags also increase, but the relationship is more prominent for small bags than large bags.
    - When total bags increase, extra large bags show a decreasing trend. 


## Question 7
Using Plotly, draw weekly and monthly time-series graphs of the numeric fields. Discuss your results.

In [ ]:
# Check for type to confirm
df_c4['date_w'].dtype

In [ ]:
# Sort by date
df_c4=df_c4.sort_values('date_w')
df_c4.head()

### Weekly time-series graphs

In [ ]:
# Resample data to a weekly frequency
df_weekly = df_c4.resample('W-Mon', on='date_w').sum().reset_index()

# Select numeric columns
numeric_col=df_c4.select_dtypes(include=['number']).columns

# Draw line graphs for each numeric column
for column in numeric_col:
    fig = px.line(df_c4, x='date_w', y=column, title=f'Weekly variation of {column}')
    fig.update_layout(xaxis_title='Date', yaxis_title=column)
    fig.show()
    

### Monthly time-series graphs

In [ ]:
# Resample data to a monthly frequency
df_monthly = df_c4.resample('ME', on='date_w').sum().reset_index()

numeric_col=df_c4.select_dtypes(include=['number']).columns

for column in numeric_col:
    fig = px.line(df_c4, x='date_w', y=column, title=f'Monthly variation of {column}')
    fig.update_layout(xaxis_title='Month', yaxis_title=column)
    fig.show()

### Discussion:

## Question 8
Draw year based location and type bar charts for the total volume using Plotly. Discuss your results.
### Year based location bar chart for total volume

In [ ]:

# Create a bar chart with multi-level x axis
fig = go.Figure()

for location in df_c4['location'].unique():
    df_location = df_c4[df_c4['location'] == location]
    fig.add_trace(go.Bar(
    x=df_location['year'].astype(str),
    y=df_location['total_vol'],
    name=location,
    ))
    

#Update layout to show multi-level x-axis
fig.update_layout(
    barmode='group',
    xaxis_title='year',
    yaxis_title='Total volume',
    title='Total volume by year and location',
    xaxis={'categoryorder':'category ascending'},
    )

fig.show()

### Year based type bar chart for total volume

In [ ]:

# Create a bar chart with multi-level x axis

fig = go.Figure()

for type in df_c4['type'].unique():
    df_type = df_c4[df_c4['type'] == type]
    fig.add_trace(go.Bar(
    x=df_type['year'].astype(str),
    y=df_type['total_vol'],
    name=type,
    ))
    

#Update layout to show multi-level x-axis
fig.update_layout(
    barmode='group',
    xaxis_title='year',
    yaxis_title='Total volume',
    title='Total volume by year and type',
    xaxis={'categoryorder':'category ascending'},
    )

fig.show()

        

### Discussion:

## Question 9
Compare and contrast the prices of each type, each location and {location and type} combination. Visualise the results using suitable plots.
### Price vs. type

In [ ]:
px.scatter(df_c4, x='type', y='price')

### Price vs. location

In [ ]:
px.scatter(df_c4, x='location', y='price')

### Price vs. location and type

In [ ]:
px.scatter(df_c4, x='location', y='price', color='type')

## Question 10
Install pandas_profiling package if it is not already installed. Create and display a profile report of df_product_a with pandas_profiling. (Note: you have to learn how to generate statistics with and without pandas_profiling)

In [ ]:
df_numeric.dtypes

In [ ]:
# Generate the profile report
profile = ProfileReport(
    df_numeric,
    title='ProductA_ProfileReport',
    #correlations=False
)

# Save report to a html file
profile.to_file('Report.html')

## Question 11
Install and try dtale https://pypi.org/project/dtale/ package.

In [ ]:
# Install dtale 
#!pip install dtale

In [ ]:
# To verify installation
#!pip show dtale

In [ ]:
# Import dtale
import dtale

In [ ]:
#Launch D-Tale
dtale_app = dtale.show(df_c4)
dtale_app

## Question 12
Use https://github.com/DataDisca/geo_coordinates to retrieve longitudes and latitudes from Google, Here and ArcGIS map service providers. Record the following.
Map Service Provider	Address	Longitude	Latitude	Response Time (ms)	Success (1 = True, 0 = False)

### Question 12.1
Copy your data into geo_retrieval_stat_<your_git_username>.csv.
The header names should be msp, address, longitude, latitude, response_time, success. Push that into your GitHub repository.